In [ ]:
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import pandas as pd 
import random
import math
import time
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error
import datetime
import operator
import matplotlib
plt.style.use('fivethirtyeight')
font = {'family' : 'serif',
        'size'   : 100}

matplotlib.rc('font', **font)
%matplotlib inline

In [ ]:
matplotlib.__version__

In [ ]:
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
deaths_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
recoveries_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')

In [ ]:
confirmed_df[confirmed_df['Country/Region']=='India'].head()

In [ ]:
deaths_df[deaths_df['Country/Region']=='India'].head()

In [ ]:
recoveries_df[recoveries_df['Country/Region']=='India'].head()

In [ ]:
cols = confirmed_df.keys()
confirmed = confirmed_df.loc[:, cols[4]:cols[-6]]
deaths = deaths_df.loc[:, cols[4]:cols[-6]]
recoveries = recoveries_df.loc[:, cols[4]:cols[-6]]

In [ ]:
dates = confirmed.keys()
total_cases = []
total_deaths = [] 
total_recovered = []
total_active = [] 

mortality_rate = []
recovery_rate = []  

india_deaths = []
india_recov = [] 
india_cases = [] 

india_mortality_rate = []
india_recovery_rate = []
india_discover_rate = []

us_deaths = []
us_recov = [] 
us_cases = [] 

italy_deaths = []
italy_recov = [] 
italy_cases = [] 

spain_deaths = []
spain_recov = [] 
spain_cases = [] 

china_deaths = []
china_recov = [] 
china_cases = [] 

In [ ]:
dates

In [ ]:
dates[0][0:dates[0].find('/')]

In [ ]:
day = 0
for i in dates:
    day=day+1
    confirmed_sum = confirmed[i].sum()
    death_sum = deaths[i].sum()
    recovered_sum = recoveries[i].sum()
    
    # confirmed, deaths, recovered, and active
    total_cases.append(confirmed_sum)
    total_deaths.append(death_sum)
    total_recovered.append(recovered_sum)
    total_active.append(confirmed_sum-death_sum-recovered_sum)
    
    # calculate rates
    mortality_rate.append(death_sum/confirmed_sum)
    recovery_rate.append(recovered_sum/confirmed_sum)

    # case studies 
    i_deaths = deaths_df[deaths_df['Country/Region']=='India'][i].sum()
    i_recov = recoveries_df[recoveries_df['Country/Region']=='India'][i].sum()
    i_cases = confirmed_df[confirmed_df['Country/Region']=='India'][i].sum()
    i_discovery = (i_cases/day)*100
    
    us_deaths.append(deaths_df[deaths_df['Country/Region']=='US'][i].sum())
    us_recov.append(recoveries_df[recoveries_df['Country/Region']=='US'][i].sum())
    us_cases.append(confirmed_df[confirmed_df['Country/Region']=='US'][i].sum())
    
    italy_deaths.append(deaths_df[deaths_df['Country/Region']=='Italy'][i].sum())
    italy_recov.append(recoveries_df[recoveries_df['Country/Region']=='Italy'][i].sum())
    italy_cases.append(confirmed_df[confirmed_df['Country/Region']=='Italy'][i].sum())
    
    spain_deaths.append(deaths_df[deaths_df['Country/Region']=='Spain'][i].sum())
    spain_recov.append(recoveries_df[recoveries_df['Country/Region']=='Spain'][i].sum())
    spain_cases.append(confirmed_df[confirmed_df['Country/Region']=='Spain'][i].sum())
    
    china_deaths.append(deaths_df[deaths_df['Country/Region']=='China'][i].sum())
    china_recov.append(recoveries_df[recoveries_df['Country/Region']=='China'][i].sum())
    china_cases.append(confirmed_df[confirmed_df['Country/Region']=='China'][i].sum())
    
    i_mortality = (i_deaths/i_cases)*100
    if i_recov == 0 and i_cases == 0:
        i_recovery = 100
    else:
        i_recovery = (i_recov/i_cases)*100
    india_cases.append(i_cases)
    india_deaths.append(i_deaths)
    india_recov.append(i_recov)
    india_mortality_rate.append(i_mortality)
    india_recovery_rate.append(i_recovery)
    india_discover_rate.append(i_discovery)

In [ ]:
d = total_deaths[-1]
r = total_recovered[-1]
c = total_cases[-1]
t = d+r+c
print(d/t,r/t,c/t)

In [ ]:
temp = []
for i in range(len(total_cases)):
    temp.append(i+1)

In [ ]:
plt.figure(figsize=(10, 3))
plt.bar(temp, total_cases, color = 'blue',width=1)
plt.plot(temp, total_cases, color='blue')
plt.bar(temp, total_recovered, color='green', width=1)
plt.plot(temp, total_recovered, color = 'green')
plt.bar(temp, total_deaths, color='red',width=1)
plt.plot(temp, total_deaths, color='red')
plt.axes().set_facecolor('white')
plt.grid(True)
plt.legend(['World cases','World recovered cases','World deaths'])
plt.savefig('plot.png', bbox_inches='tight', dpi=400, facecolor='white')

In [ ]:
india_cases = np.array(india_cases).reshape(-1, 1)
india_deaths = np.array(india_deaths).reshape(-1, 1)
india_recov = np.array(india_recov).reshape(-1, 1)
mortality_rate = np.array(mortality_rate).reshape(-1, 1)
recovery_rate = np.array(recovery_rate).reshape(-1, 1)
india_mortality_rate = np.array(india_mortality_rate).reshape(-1, 1)
india_recovery_rate = np.array(india_recovery_rate).reshape(-1, 1)
india_discover_rate = np.array(india_discover_rate).reshape(-1, 1)

In [ ]:
fig = plt.figure(figsize=(18,10))
fig.subplots_adjust(hspace=0.2, wspace=0.2)
ax = fig.add_subplot(2, 1, 1)
ax.plot(temp, india_cases, '-', label='India cases',linewidth=2, markersize=5, color='red')
ax.plot(india_recov, '-', label='India recovered',linewidth=2, markersize=5, color='green')
ax.plot(india_deaths, '-', label='India deaths',linewidth=2, markersize=5, color='blue')
ax.tick_params(axis ='x', rotation =90, width =50)
ax.legend()

ax2 = fig.add_subplot(2, 1, 2)
ax2.plot(temp,india_recovery_rate/100, '-', label='India recovery rate',linewidth=2, markersize=5, color='cyan')
ax2.plot(india_mortality_rate/100, '-', label='India death rate',linewidth=2, markersize=5, color='yellow')
ax2.plot(india_discover_rate/100, '-', label='India death rate',linewidth=2, markersize=5, color='brown')
ax2.tick_params(axis ='x', rotation =90, width =50)
ax2.legend()

In [ ]:
def daily_increase(data):
    d = [] 
    for i in range(len(data)):
        if i == 0:
            d.append(data[0])
        else:
            d.append(data[i]-data[i-1])
    return d 

In [ ]:
total_cases_increase = daily_increase(total_cases)
total_deaths_increase = daily_increase(total_deaths)
total_recovered_increase = daily_increase(total_recovered)

In [ ]:
india_daily_increase = daily_increase(india_cases)
india_deaths_increase = daily_increase(india_deaths)
india_recov_increase = daily_increase(india_recov)

In [ ]:
days_in_future = 10
future_forcast = np.array([i for i in range(len(dates)+days_in_future)]).reshape(-1, 1)
adjusted_dates = future_forcast[:-days_in_future]

In [ ]:
start = '1/22/2020'
start_date = datetime.datetime.strptime(start, '%m/%d/%Y')
future_forcast_dates = []
for i in range(len(future_forcast)):
    future_forcast_dates.append((start_date + datetime.timedelta(days=i)).strftime('%m/%d/%Y'))

In [ ]:
for i in future_forcast_dates:
    print(i,end=',')

In [ ]:
x_train = adjusted_dates
y_train = total_cases
offset = y_train[-1]

In [ ]:
svm_confirmed_world = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=6, C=0.1)
svm_confirmed_world.fit(x_train, y_train)
svm_pred = svm_confirmed_world.predict(future_forcast)

In [ ]:
#svm_test_pred = svm_confirmed.predict(X_test_confirmed)
plt.plot(y_train)
plt.plot(future_forcast[:-days_in_future-1],svm_pred[:-days_in_future-1],'--', color='red')
plt.plot(future_forcast[-days_in_future-1:],svm_pred[-days_in_future-1:], color='red')
#plt.legend(['Original', 'SVR Predictions'])
#print('MAE:', mean_absolute_error(svm_test_pred, y_test_confirmed))
plt.axes().set_facecolor('white')
print('MSE:',mean_squared_error(svm_pred[:-days_in_future],np.array(y_train)))

In [ ]:
svm_pred[-1]

In [ ]:
x_train = adjusted_dates
y_train = total_deaths

In [ ]:
svm_deaths_world = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=6, C=0.1)
svm_deaths_world.fit(x_train, y_train)
svm_pred = svm_deaths_world.predict(future_forcast)

In [ ]:
#svm_test_pred = svm_confirmed.predict(X_test_confirmed)
plt.plot(y_train)
plt.plot(future_forcast[:-days_in_future-1],svm_pred[:-days_in_future-1],'--', color='red')
plt.plot(future_forcast[-days_in_future-1:],svm_pred[-days_in_future-1:], color='red')
#plt.legend(['Original', 'SVR Predictions'])
#print('MAE:', mean_absolute_error(svm_test_pred, y_test_confirmed))
plt.axes().set_facecolor('white')
print('MSE:',mean_squared_error(svm_pred[:-days_in_future],np.array(y_train)))

In [ ]:
svm_pred[-1]

In [ ]:
x_train = adjusted_dates
y_train = total_recovered
svm_recov_world = SVR(shrinking=True, kernel='poly',gamma=0.01, epsilon=1,degree=6, C=0.1)
svm_recov_world.fit(x_train, y_train)
svm_pred = svm_recov_world.predict(future_forcast)
#svm_test_pred = svm_confirmed.predict(X_test_confirmed)
plt.plot(y_train)
plt.plot(future_forcast[:-days_in_future-1],svm_pred[:-days_in_future-1],'--', color='red')
plt.plot(future_forcast[-days_in_future-1:],svm_pred[-days_in_future-1:], color='red')
#plt.legend(['Original', 'SVR Predictions'])
plt.axes().set_facecolor('white')
#print('MAE:', mean_absolute_error(svm_test_pred, y_test_confirmed))
print('MSE:',mean_squared_error(svm_pred[:-days_in_future],np.array(y_train)))

In [ ]:
svm_pred[-1]

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten

In [ ]:
x_train = adjusted_dates
y_train = np.array(total_cases)
offet = y_train[-1]

In [ ]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = x_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='relu'))

# Compile the network :
NN_model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

In [ ]:
earlystopper = EarlyStopping(patience=200, verbose=1)
checkpointer = ModelCheckpoint('model-unet-inception-15.h5', verbose=1, save_best_only=True)
NN_model.fit(x_train, y_train, epochs=2000, batch_size=32, validation_split = 0.2, callbacks=[earlystopper, checkpointer])

In [ ]:
predictions = NN_model.predict(future_forcast)
predictions[-1]

In [ ]:
plt.plot(y_train)
plt.plot(future_forcast[:-days_in_future-1],predictions[:-days_in_future-1],'--', color='red')
plt.plot(future_forcast[-days_in_future-1:],predictions[-days_in_future-1:], color='red')
#plt.legend(['Original', 'DNN Predictions'])
plt.axes().set_facecolor('white')
#print('MAE:', mean_absolute_error(svm_test_pred, y_test_confirmed))
print('MSE:',mean_squared_error(predictions[:-days_in_future],np.array(y_train)))

In [ ]:
x_train = adjusted_dates
y_train = np.array(total_deaths)
NN_model2 = Sequential()

# The Input Layer :
NN_model2.add(Dense(128, kernel_initializer='normal',input_dim = x_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model2.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model2.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model2.add(Dense(256, kernel_initializer='normal',activation='relu'))
# The Output Layer :
NN_model2.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model2.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model2.summary()
earlystopper = EarlyStopping(patience=200, verbose=1)
checkpointer = ModelCheckpoint('model-unet-inception-15.h5', verbose=1, save_best_only=True)
NN_model2.fit(x_train, y_train, epochs=2000, batch_size=32, validation_split = 0.2, callbacks=[earlystopper, checkpointer])
predictions = NN_model2.predict(future_forcast)
print(predictions[-1])
plt.plot(y_train)
plt.plot(future_forcast[:-days_in_future-1],predictions[:-days_in_future-1],'--', color='red')
plt.plot(future_forcast[-days_in_future-1:],predictions[-days_in_future-1:], color='red')
#plt.legend(['Original', 'DNN Predictions'])
plt.axes().set_facecolor('white')
#print('MAE:', mean_absolute_error(svm_test_pred, y_test_confirmed))
print('MSE:',mean_squared_error(predictions[:-days_in_future],np.array(y_train)))

In [ ]:
x_train = adjusted_dates
y_train = np.array(total_recovered)
NN_model3 = Sequential()

# The Input Layer :
NN_model3.add(Dense(128, kernel_initializer='normal',input_dim = x_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model3.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model3.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model3.add(Dense(256, kernel_initializer='normal',activation='relu'))
# The Output Layer :
NN_model3.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model3.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model3.summary()
earlystopper = EarlyStopping(patience=200, verbose=1)
checkpointer = ModelCheckpoint('model-unet-inception-15.h5', verbose=1, save_best_only=True)
NN_model3.fit(x_train, y_train, epochs=2000, batch_size=32, validation_split = 0.2, callbacks=[earlystopper, checkpointer])
predictions = NN_model3.predict(future_forcast)
print(predictions[-1])
plt.plot(y_train)
plt.plot(future_forcast[:-days_in_future-1],predictions[:-days_in_future-1],'--', color='red')
plt.plot(future_forcast[-days_in_future-1:],predictions[-days_in_future-1:], color='red')
#plt.legend(['Original', 'DNN Predictions'])
plt.axes().set_facecolor('white')
#print('MAE:', mean_absolute_error(svm_test_pred, y_test_confirmed))
print('MSE:',mean_squared_error(predictions[:-days_in_future],np.array(y_train)))

In [ ]:
x_train = adjusted_dates
y_train = np.array(total_cases)

In [ ]:
# transform our data for polynomial regression
poly = PolynomialFeatures(degree=6)
poly_X_train_confirmed = poly.fit_transform(x_train)
poly_future_forcast = poly.fit_transform(future_forcast)

In [ ]:
linear_model = LinearRegression(normalize=True, fit_intercept=False)
linear_model.fit(poly_X_train_confirmed, y_train)

In [ ]:
test_linear_pred = linear_model.predict(poly_future_forcast)
test_linear_pred[-1]

In [ ]:
plt.plot(y_train)
plt.plot(future_forcast[:-days_in_future-1],test_linear_pred[:-days_in_future-1],'--', color='red')
plt.plot(future_forcast[-days_in_future-1:],test_linear_pred[-days_in_future-1:], color='red')
#plt.legend(['Original', 'PR Predictions'])
#print('MAE:', mean_absolute_error(svm_test_pred, y_test_confirmed))
plt.axes().set_facecolor('white')
print('MSE:',mean_squared_error(test_linear_pred[:-days_in_future],np.array(y_train)))

In [ ]:
x_train = adjusted_dates
y_train = np.array(total_deaths)
poly = PolynomialFeatures(degree=6)
poly_X_train_confirmed = poly.fit_transform(x_train)
poly_future_forcast = poly.fit_transform(future_forcast)
linear_model = LinearRegression(normalize=True, fit_intercept=False)
linear_model.fit(poly_X_train_confirmed, y_train)
test_linear_pred = linear_model.predict(poly_future_forcast)
print(test_linear_pred[-1])
plt.plot(y_train)
plt.plot(future_forcast[:-days_in_future-1],test_linear_pred[:-days_in_future-1],'--', color='red')
plt.plot(future_forcast[-days_in_future-1:],test_linear_pred[-days_in_future-1:], color='red')
#plt.legend(['Original', 'PR Predictions'])
plt.axes().set_facecolor('white')
#print('MAE:', mean_absolute_error(svm_test_pred, y_test_confirmed))
print('MSE:',mean_squared_error(test_linear_pred[:-days_in_future],np.array(y_train)))

In [ ]:
x_train = adjusted_dates
y_train = np.array(total_recovered)
poly = PolynomialFeatures(degree=6)
poly_X_train_confirmed = poly.fit_transform(x_train)
poly_future_forcast = poly.fit_transform(future_forcast)
linear_model = LinearRegression(normalize=True, fit_intercept=False)
linear_model.fit(poly_X_train_confirmed, y_train)
test_linear_pred = linear_model.predict(poly_future_forcast)
print(test_linear_pred[-1])
plt.plot(y_train)
plt.plot(future_forcast[:-days_in_future-1],test_linear_pred[:-days_in_future-1],'--', color='red')
plt.plot(future_forcast[-days_in_future-1:],test_linear_pred[-days_in_future-1:], color='red')
#plt.legend(['Original', 'PR Predictions'])
plt.axes().set_facecolor('white')
#print('MAE:', mean_absolute_error(svm_test_pred, y_test_confirmed))
print('MSE:',mean_squared_error(test_linear_pred[:-days_in_future],np.array(y_train)))

In [ ]:
from keras.layers import Dense
from keras.layers import LSTM, Dropout

In [ ]:
x_train = adjusted_dates
y_train = np.array(total_cases)
offset=y_train[-1]

In [ ]:
y_train = (MinMaxScaler().fit_transform(np.array(y_train).reshape(-1,1))).reshape(-1,1)

In [ ]:
model = Sequential()
model.add(LSTM(256, return_sequences=False, input_shape=(1,1)))
model.add(Dropout(0.1))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')
#model.fit(trainX, trainY, epochs=1000, batch_size=240, verbose=1)

In [ ]:
model.summary()

In [ ]:
X_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))
X_test = np.reshape(y_train, (y_train.shape[0], 1))
predict = np.reshape(future_forcast, (future_forcast.shape[0], future_forcast.shape[1],1))

In [ ]:
earlystopper = EarlyStopping(patience=200, verbose=1)
model.fit(X_train, X_test, epochs=5000, batch_size=64, verbose=1, callbacks=[earlystopper])

In [ ]:
test_linear_pred = model.predict(predict)

In [ ]:
y_train=y_train*offset
test_linear_pred = test_linear_pred * offset
#test_linear_pred[-days_in_future-1:] = test_linear_pred[-days_in_future-1:] * 2

In [ ]:
plt.plot(y_train)
plt.plot(future_forcast[:-days_in_future-1],test_linear_pred[:-days_in_future-1],'--', color='red')
plt.plot(future_forcast[-days_in_future-1:],test_linear_pred[-days_in_future-1:], color='red')
#plt.legend(['Original', 'LSTM Predictions'])
plt.axes().set_facecolor('white')
#print('MAE:', mean_absolute_error(svm_test_pred, y_test_confirmed))
print('MSE:',mean_squared_error(test_linear_pred[:-days_in_future],np.array(y_train)))

In [ ]:
test_linear_pred[-1]

In [ ]:
x_train = adjusted_dates
y_train = np.array(total_deaths)
offset = y_train[-1]
y_train = (MinMaxScaler().fit_transform(np.array(y_train).reshape(-1,1))).reshape(-1,1)
model2 = Sequential()
model2.add(LSTM(256, return_sequences=False, input_shape=(1,1)))
model2.add(Dropout(0.1))
model2.add(Dense(1))
model2.compile(loss='mse', optimizer='adam')
model2.summary()
X_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))
X_test = np.reshape(y_train, (y_train.shape[0], 1))
predict = np.reshape(future_forcast, (future_forcast.shape[0], future_forcast.shape[1],1))
model2.fit(X_train, X_test, epochs=5000, batch_size=64, verbose=1)

In [ ]:
test_linear_pred = model2.predict(predict)
y_train=y_train*offset
test_linear_pred = test_linear_pred * offset
plt.plot(y_train)
plt.plot(future_forcast[:-days_in_future-1],test_linear_pred[:-days_in_future-1],'--', color='red')
plt.plot(future_forcast[-days_in_future-1:],test_linear_pred[-days_in_future-1:], color='red')
#plt.legend(['Original', 'LSTM Predictions'])
plt.axes().set_facecolor('white')
#print('MAE:', mean_absolute_error(svm_test_pred, y_test_confirmed))
print('MSE:',mean_squared_error(test_linear_pred[:-days_in_future],np.array(y_train)))

In [ ]:
test_linear_pred[-1]

In [ ]:
x_train = adjusted_dates
y_train = np.array(total_recovered)
offset = y_train[-1]
y_train = (MinMaxScaler().fit_transform(np.array(y_train).reshape(-1,1))).reshape(-1,1)
model3 = Sequential()
model3.add(LSTM(256, return_sequences=False, input_shape=(1,1)))
model3.add(Dropout(0.1))
model3.add(Dense(1))
model3.compile(loss='mse', optimizer='adam')
model3.summary()
X_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1],1))
X_test = np.reshape(y_train, (y_train.shape[0], 1))
predict = np.reshape(future_forcast, (future_forcast.shape[0], future_forcast.shape[1],1))
model3.fit(X_train, X_test, epochs=10000, batch_size=64, verbose=1)

In [ ]:
test_linear_pred = model3.predict(predict)
y_train=y_train*offset
test_linear_pred = test_linear_pred * offset
plt.plot(y_train)
plt.plot(future_forcast[:-days_in_future-1],test_linear_pred[:-days_in_future-1],'--', color='red')
plt.plot(future_forcast[-days_in_future-1:],test_linear_pred[-days_in_future-1:], color='red')
#plt.legend(['Original', 'LSTM Predictions'])
plt.axes().set_facecolor('white')
#print('MAE:', mean_absolute_error(svm_test_pred, y_test_confirmed))
print('MSE:',mean_squared_error(test_linear_pred[:-days_in_future],np.array(y_train)))

In [ ]:
test_linear_pred[-1]